In [11]:
%pip install sqlite3 sentence_transformers numpy

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [15]:
import sqlite3
from sentence_transformers import SentenceTransformer
import numpy as np

model_name = 'sentence-transformers/msmarco-distilbert-base-v3'

# Connect to the SQLite database
conn = sqlite3.connect('scriptures.db')
cursor = conn.cursor()

# Create the verse_chunks table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS verse_chunks (
    id INTEGER PRIMARY KEY,
    verse_id INTEGER REFERENCES verses(id) ON DELETE CASCADE,
    chunk_text TEXT,
    embedding BLOB
)
''')

# Load the embedding model
model = SentenceTransformer(model_name)

# Function to chunk text into smaller pieces
def chunk_text(text, max_length=32):
    words = text.split()
    return [' '.join(words[i:i + max_length]) for i in range(0, len(words), max_length)]

# Fetch all verses
cursor.execute("SELECT id, scripture_text FROM verses")
verses = cursor.fetchall()

# Process each verse
for verse_id, scripture_text in verses:
    chunks = chunk_text(scripture_text)
    for chunk in chunks:
        embedding = model.encode(chunk)
        embedding_blob = sqlite3.Binary(np.array(embedding).tobytes())
        
        # Insert chunk and embedding into the database
        cursor.execute("INSERT INTO verse_chunks (verse_id, chunk_text, embedding) VALUES (?, ?, ?)", 
                       (verse_id, chunk, embedding_blob))

# Commit the changes and close the connection
conn.commit()
conn.close()


In [2]:
import sqlite3
from sentence_transformers import SentenceTransformer
import numpy as np

model_name = 'sentence-transformers/msmarco-distilbert-base-v3'

# Connect to the SQLite database
conn = sqlite3.connect('scriptures.db')
cursor = conn.cursor()

# Create the verse_chunks table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS verse_chunks (
    id INTEGER PRIMARY KEY,
    verse_id INTEGER REFERENCES verses(id) ON DELETE CASCADE,
    chunk_text TEXT,
    embedding BLOB
)
''')

# Load the embedding model
model = SentenceTransformer(model_name)

# Function to chunk text into overlapping pieces
def chunk_text(text, max_length=32, overlap=16):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_length - overlap):
        chunk = ' '.join(words[i:i + max_length])
        if chunk:
            chunks.append(chunk)
    return chunks

# Fetch all verses
cursor.execute("SELECT id, scripture_text FROM verses")
verses = cursor.fetchall()

# Process each verse
for verse_id, scripture_text in verses:
    chunks = chunk_text(scripture_text)
    for chunk in chunks:
        embedding = model.encode(chunk)
        embedding_blob = sqlite3.Binary(np.array(embedding).tobytes())
        
        # Insert chunk and embedding into the database
        cursor.execute("INSERT INTO verse_chunks (verse_id, chunk_text, embedding) VALUES (?, ?, ?)", 
                       (verse_id, chunk, embedding_blob))

# Commit the changes and close the connection
conn.commit()
conn.close()


c:\Users\caleb\code\versio\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
import sqlite3
import math
from collections import defaultdict, Counter
import string

# Connect to SQLite database
connection = sqlite3.connect('scriptures.db')

# Create bm25_scores table
create_table_query = """
CREATE TABLE IF NOT EXISTS bm25_scores (
    verse_id INTEGER REFERENCES verses(id) ON DELETE CASCADE,
    term TEXT,
    score REAL,
    PRIMARY KEY (verse_id, term)
);
"""
cursor = connection.cursor()
cursor.execute(create_table_query)
connection.commit()

# Close the connection
connection.close()

print("Table 'bm25_scores' has been created successfully.")

class BM25:
    def __init__(self, k1=1.5, b=0.75, avg_doc_length=100.0):
        self.k1 = k1
        self.b = b
        self.avg_doc_length = avg_doc_length

    def calculate_score(self, doc_length, term_frequency, doc_frequency, total_docs):
        idf = math.log((total_docs - doc_frequency + 0.5) / (doc_frequency + 0.5) + 1)
        tf = (term_frequency * (self.k1 + 1)) / (term_frequency + self.k1 * (1 - self.b + self.b * (doc_length / self.avg_doc_length)))
        return idf * tf

def tokenize(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.lower().translate(translator).split()

def compute_bm25_scores(connection):
    bm25 = BM25()

    cursor = connection.cursor()
    
    # Get total number of documents
    cursor.execute("SELECT COUNT(*) FROM verses")
    total_docs = cursor.fetchone()[0]

    # Calculate average document length
    cursor.execute("SELECT AVG(LENGTH(scripture_text)) FROM verses")
    avg_doc_length = cursor.fetchone()[0]
    bm25.avg_doc_length = avg_doc_length

    # Get document frequency for each term
    cursor.execute("SELECT id, scripture_text FROM verses")
    verses = cursor.fetchall()

    doc_frequencies = defaultdict(int)
    term_frequencies = defaultdict(Counter)
    doc_lengths = {}

    for verse_id, scripture_text in verses:
        terms = tokenize(scripture_text)
        doc_lengths[verse_id] = len(terms)
        term_frequencies[verse_id].update(terms)
        for term in set(terms):
            doc_frequencies[term] += 1

    # Clear existing bm25_scores
    cursor.execute("DELETE FROM bm25_scores")

    # Insert BM25 scores
    for verse_id, terms in term_frequencies.items():
        doc_length = doc_lengths[verse_id]
        for term, term_frequency in terms.items():
            doc_frequency = doc_frequencies[term]
            score = bm25.calculate_score(doc_length, term_frequency, doc_frequency, total_docs)
            cursor.execute("INSERT INTO bm25_scores (verse_id, term, score) VALUES (?, ?, ?)", (verse_id, term, score))

    connection.commit()

# Connect to SQLite database
connection = sqlite3.connect('scriptures.db')

# Compute and store BM25 scores
compute_bm25_scores(connection)

# Close the connection
connection.close()

print("BM25 scores have been successfully computed and stored.")



Table 'bm25_scores' has been created successfully.
BM25 scores have been successfully computed and stored.


In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

with open("vocab.txt", "w") as f:
    for token, id in tokenizer.vocab.items():
        f.write(f"{token}\t{id}\n")

c:\Users\caleb\code\versio\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


UnicodeEncodeError: 'charmap' codec can't encode character '\u4f8d' in position 0: character maps to <undefined>